# 🚀 FSKU 데이터 생성 파이프라인

고성능 환경에서 금융보안 데이터를 대량 생성하고 체이닝으로 품질을 극대화하는 노트북입니다.

## 📋 목차
1. 환경 설정 및 패키지 설치
2. 기존 모듈 임포트 및 설정
3. 체이닝 기능이 추가된 DataGenerator
4. RAG 시스템 초기화
5. 프롬프트 체이닝 템플릿
6. 대규모 배치 생성
7. 품질 모니터링 대시보드
8. 메인 실행 및 데이터 저장

## 1. 환경 설정 및 패키지 설치

In [ ]:
# GPU 환경 확인
!nvidia-smi

# 필수 패키지 설치 (고성능 환경용)
!pip install -q transformers accelerate bitsandbytes
!pip install -q langchain sentence-transformers faiss-gpu
!pip install -q PyPDF2 pdfplumber pandas tqdm matplotlib
!pip install -q flash-attn --no-build-isolation  # Flash Attention 2
!pip install -q datasets tokenizers

## 2. 기존 모듈 임포트 및 설정

In [ ]:
import sys
import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Optional, Any
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 프로젝트 경로 설정
PROJECT_ROOT = Path('/workspace/ai-dacon')  # RunPod 환경에 맞게 수정
sys.path.append(str(PROJECT_ROOT / 'src'))

# 기존 모듈 임포트
from rag.retriever import DocumentRetriever
from rag.pdf_loader import DocumentLoader
from rag.chunker import DocumentChunker
from generate_data.main import DataGenerator
from generate_data.quality_checker import QualityChecker

print("✅ 모듈 임포트 완료")
print(f"🔧 프로젝트 경로: {PROJECT_ROOT}")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"💻 CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   - GPU {i}: {torch.cuda.get_device_name(i)}")

In [ ]:
# 설정 정의 (리소스 제약 없음)
config = {
    # 모델 설정
    'model_name': 'Qwen/Qwen2.5-72B-Instruct',  # 메인 생성 모델 (대형)
    'validation_model': 'meta-llama/Llama-3-70B-Instruct',  # 검증용 모델
    'use_quantization': False,  # 양자화 사용 안함 (전체 정밀도)
    'device_map': 'auto',  # 멀티 GPU 자동 분배
    
    # 생성 설정
    'num_questions': 10000,  # 목표 생성 수
    'batch_size': 32,  # 대규모 배치
    'temperature': 0.8,
    'top_p': 0.9,
    'max_new_tokens': 1024,
    
    # RAG 설정
    'use_rag': True,
    'search_method': 'hybrid',  # hybrid, similarity, bm25
    'top_k_retrieval': 5,
    
    # 체이닝 설정
    'use_chaining': True,
    'chain_steps': ['generate', 'validate', 'refine', 'final_check'],
    'min_quality_score': 80,
    
    # 저장 설정
    'output_dir': PROJECT_ROOT / 'data' / 'generated',
    'checkpoint_interval': 1000,  # 1000개마다 체크포인트
    
    # 모니터링
    'enable_monitoring': True,
    'update_interval': 100  # 100개마다 대시보드 업데이트
}

# 출력 디렉토리 생성
config['output_dir'].mkdir(parents=True, exist_ok=True)
print(f"📁 출력 디렉토리: {config['output_dir']}")

## 3. 체이닝 기능이 추가된 DataGenerator

In [ ]:
class ChainDataGenerator(DataGenerator):
    """체이닝 기능이 추가된 데이터 생성기"""
    
    def __init__(self, config: Dict[str, Any]):
        """초기화"""
        super().__init__(config)
        self.chain_steps = config.get('chain_steps', ['generate'])
        self.validation_model = None
        self.validation_tokenizer = None
        
        # 검증용 모델 로드
        if config.get('validation_model') and 'validate' in self.chain_steps:
            self._load_validation_model()
    
    def _load_validation_model(self):
        """검증용 모델 로드"""
        print(f"🔄 검증 모델 로딩: {self.config['validation_model']}")
        
        self.validation_tokenizer = AutoTokenizer.from_pretrained(
            self.config['validation_model'],
            trust_remote_code=True
        )
        
        if self.validation_tokenizer.pad_token is None:
            self.validation_tokenizer.pad_token = self.validation_tokenizer.eos_token
        
        # 전체 정밀도로 로드 (리소스 제약 없음)
        self.validation_model = AutoModelForCausalLM.from_pretrained(
            self.config['validation_model'],
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )
        
        print("✅ 검증 모델 로드 완료")
    
    def generate_with_chain(self, prompt: str, context: str) -> Dict[str, Any]:
        """체이닝을 통한 문제 생성"""
        result = {
            'prompt': prompt,
            'context': context,
            'chain_history': []
        }
        
        current_output = None
        
        for step in self.chain_steps:
            if step == 'generate':
                # 1단계: 초기 생성
                current_output = self._generate_with_llm(prompt, self.config['temperature'])
                result['initial_output'] = current_output
                
            elif step == 'validate' and self.validation_model:
                # 2단계: 검증
                validation_result = self._validate_output(current_output)
                result['validation_score'] = validation_result['score']
                result['validation_feedback'] = validation_result['feedback']
                
                if validation_result['score'] < self.config['min_quality_score']:
                    current_output = validation_result['suggestion']
                    
            elif step == 'refine':
                # 3단계: 개선
                refined_output = self._refine_output(current_output, result.get('validation_feedback', ''))
                current_output = refined_output
                
            elif step == 'final_check':
                # 4단계: 최종 검증
                final_score = self.quality_checker.evaluate(current_output)
                result['final_score'] = final_score
            
            result['chain_history'].append({
                'step': step,
                'output': current_output[:200] + '...' if len(current_output) > 200 else current_output
            })
        
        result['final_output'] = current_output
        return result
    
    def _validate_output(self, output: str) -> Dict[str, Any]:
        """검증 모델을 사용한 출력 평가"""
        validation_prompt = f"""다음 금융 시험 문제를 평가하고 개선점을 제시하세요.

문제:
{output}

평가 기준:
1. 명확성 (1-10): 문제가 명확하고 모호하지 않은가?
2. 정확성 (1-10): 금융 지식이 정확한가?
3. 난이도 (1-10): 적절한 난이도인가?
4. 형식 (1-10): 문제 형식이 올바른가?

출력 형식:
[점수] 총점: XX/40
[피드백] 개선이 필요한 부분
[제안] 개선된 문제
"""
        
        # 검증 모델로 평가
        inputs = self.validation_tokenizer(
            validation_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(self.validation_model.device)
        
        with torch.no_grad():
            outputs = self.validation_model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.7,
                do_sample=True
            )
        
        validation_output = self.validation_tokenizer.decode(outputs[0], skip_special_tokens=True)
        validation_output = validation_output.replace(validation_prompt, '').strip()
        
        # 결과 파싱
        import re
        score_match = re.search(r'총점[:\s]*(\d+)', validation_output)
        score = int(score_match.group(1)) if score_match else 20
        
        feedback_match = re.search(r'\[피드백\](.*?)\[제안\]', validation_output, re.DOTALL)
        feedback = feedback_match.group(1).strip() if feedback_match else ''
        
        suggestion_match = re.search(r'\[제안\](.*)', validation_output, re.DOTALL)
        suggestion = suggestion_match.group(1).strip() if suggestion_match else output
        
        return {
            'score': (score / 40) * 100,  # 100점 만점으로 변환
            'feedback': feedback,
            'suggestion': suggestion
        }
    
    def _refine_output(self, output: str, feedback: str) -> str:
        """피드백을 반영한 출력 개선"""
        refine_prompt = f"""다음 금융 문제를 피드백을 참고하여 개선하세요.

원본 문제:
{output}

피드백:
{feedback}

개선된 문제 (더 명확하고 전문적으로):
"""
        
        refined = self._generate_with_llm(refine_prompt, temperature=0.7)
        return refined
    
    def generate_questions_batch(self, chunks: List[Dict], batch_size: int = 32) -> List[Dict]:
        """배치 단위로 문제 생성"""
        questions = []
        
        for i in range(0, len(chunks), batch_size):
            batch_chunks = chunks[i:i+batch_size]
            batch_prompts = []
            
            for chunk in batch_chunks:
                prompt = self.prompt_template.format(
                    concept=chunk.get('keywords', ['금융'])[0] if chunk.get('keywords') else '금융',
                    context=chunk['content']
                )
                batch_prompts.append(prompt)
            
            # 배치 처리 (현재는 순차 처리, 향후 병렬화 가능)
            for prompt, chunk in zip(batch_prompts, batch_chunks):
                try:
                    if self.config.get('use_chaining'):
                        result = self.generate_with_chain(prompt, chunk['content'])
                        if result['final_score'] >= self.config['min_quality_score']:
                            question = self._parse_question_result(result)
                            questions.append(question)
                    else:
                        # 기존 방식
                        generated = self._generate_with_llm(prompt, self.config['temperature'])
                        question = self._extract_question_answer(generated)
                        questions.append(question)
                except Exception as e:
                    print(f"⚠️ 생성 오류: {e}")
                    continue
        
        return questions
    
    def _parse_question_result(self, result: Dict) -> Dict:
        """체이닝 결과를 문제 형식으로 파싱"""
        final_output = result['final_output']
        
        # 기존 파싱 메서드 사용
        question = self._clean_question(final_output)
        answer = self._extract_answer(final_output)
        
        return {
            'id': f"CHAIN_{int(time.time()*1000) % 1000000:06d}",
            'question': question,
            'answer': answer,
            'metadata': {
                'chain_steps': len(result['chain_history']),
                'initial_score': result.get('validation_score', 0),
                'final_score': result.get('final_score', 0),
                'context_used': bool(result.get('context'))
            }
        }
    
    def _extract_question_answer(self, generated: str) -> Dict:
        """기존 방식의 질문/답변 추출"""
        question = self._clean_question(generated)
        answer = self._extract_answer(generated)
        
        return {
            'id': f"GEN_{int(time.time()*1000) % 1000000:06d}",
            'question': question,
            'answer': answer
        }

## 4. RAG 시스템 초기화

In [ ]:
# RAG 시스템 초기화 (기존 retriever 활용)
print("🔄 RAG 시스템 초기화 중...")

# 문서 경로 확인
data_dir = PROJECT_ROOT / 'data'
external_dir = data_dir / 'external'

if not external_dir.exists():
    print(f"⚠️ 외부 데이터 디렉토리가 없습니다: {external_dir}")
    print("PDF 파일을 업로드하거나 경로를 수정하세요.")
else:
    # DocumentRetriever 초기화
    retriever = DocumentRetriever(
        data_dir=str(data_dir),
        use_embedding=True,
        use_cache=True
    )
    
    # 통계 출력
    stats = retriever.get_statistics()
    print(f"\n📊 RAG 통계:")
    print(f"  - 문서 수: {stats['total_documents']}")
    print(f"  - 청크 수: {stats['total_chunks']}")
    print(f"  - 평균 청크 크기: {stats['avg_chunk_size']:.1f} 토큰")
    print(f"  - 임베딩 사용: {stats['use_embedding']}")
    print(f"  - 문서 타입: {stats['document_types']}")


def multi_method_search(retriever, query: str, methods: List[str] = None) -> str:
    """여러 검색 방법을 조합한 검색"""
    if methods is None:
        methods = ['similarity', 'bm25', 'hybrid']
    
    all_contexts = []
    seen_chunks = set()
    
    for method in methods:
        try:
            context = retriever.search(query, top_k=3, method=method)
            # 중복 제거
            if context and context not in seen_chunks:
                all_contexts.append(f"[{method.upper()}]\n{context}")
                seen_chunks.add(context[:100])  # 앞부분으로 중복 체크
        except Exception as e:
            print(f"⚠️ {method} 검색 실패: {e}")
    
    return "\n\n---\n\n".join(all_contexts)


# 테스트 검색
if 'retriever' in locals():
    test_query = "전자금융거래"
    test_result = multi_method_search(retriever, test_query, ['similarity', 'bm25'])
    print(f"\n🔍 테스트 검색 '{test_query}':")
    print(test_result[:500] + "..." if len(test_result) > 500 else test_result)

## 5. 프롬프트 체이닝 템플릿

In [ ]:
# 프롬프트 템플릿 정의
PROMPT_TEMPLATES = {
    'initial': """당신은 한국 금융보안원의 수석 전문가입니다.
다음 금융 문서를 바탕으로 FSKU 평가에 적합한 고품질 시험 문제를 생성하세요.

📌 문서 내용:
{context}

📋 문제 요구사항:
1. 형식: 객관식(4-5지선다) 또는 주관식
2. 난이도: 중상급 (금융 전문가 대상)
3. 내용: 실무에 직접 적용 가능한 지식
4. 명확성: 모호하지 않고 구체적인 표현

🎯 생성할 문제 유형: {question_type}

[문제]
(여기에 문제 작성)

[선택지] (객관식인 경우)
1) 
2) 
3) 
4) 
5) 

[정답]
(정답 번호 또는 서술형 답변)

[해설]
(정답의 근거와 오답 분석)""",
    
    'validation': """금융 교육 전문가로서 다음 시험 문제를 평가해주세요.

📝 평가할 문제:
{question}

🔍 평가 기준:
1. 명확성 (1-10): 문제가 명확하고 이해하기 쉬운가?
2. 정확성 (1-10): 금융 지식과 법규가 정확한가?
3. 난이도 (1-10): 전문가 수준에 적절한가?
4. 실무성 (1-10): 실제 업무에 도움이 되는가?

💡 출력 형식:
[점수] 총점: XX/40
- 명확성: X/10
- 정확성: X/10
- 난이도: X/10
- 실무성: X/10

[피드백]
개선이 필요한 구체적인 부분

[제안]
개선된 문제 전체""",
    
    'refinement': """다음 피드백을 반영하여 금융 시험 문제를 개선하세요.

📌 원본 문제:
{original}

💭 피드백:
{feedback}

✨ 개선 지침:
- 더 명확하고 구체적인 표현 사용
- 금융 전문 용어의 정확한 사용
- 실무 상황을 반영한 사례 포함
- 선택지 간 명확한 구분

📝 개선된 문제:
[문제]

[선택지]

[정답]

[해설]""",
    
    'final_check': """최종 품질 검증을 수행하세요.

문제: {question}

체크리스트:
□ 문법적 오류가 없는가?
□ 금융 용어가 정확한가?
□ 법령 조항이 최신인가?
□ 정답이 명확한가?
□ 난이도가 적절한가?

최종 평가: [통과/수정필요]"""
}

# 문제 유형 정의
QUESTION_TYPES = [
    "정의형 (용어나 개념의 정의를 묻는 문제)",
    "사례형 (실제 상황에서의 적용을 묻는 문제)",
    "계산형 (수치 계산이 필요한 문제)",
    "판단형 (옳고 그름을 판단하는 문제)",
    "비교형 (여러 개념을 비교하는 문제)",
    "법령형 (특정 법령 조항을 묻는 문제)"
]

# 프롬프트 템플릿 저장
prompt_dir = config['output_dir'] / 'prompts'
prompt_dir.mkdir(exist_ok=True)

for name, template in PROMPT_TEMPLATES.items():
    with open(prompt_dir / f'{name}_prompt.txt', 'w', encoding='utf-8') as f:
        f.write(template)

print(f"✅ {len(PROMPT_TEMPLATES)}개 프롬프트 템플릿 생성 완료")
print(f"📁 저장 위치: {prompt_dir}")

## 6. 품질 모니터링 대시보드

In [ ]:
class ProgressMonitor:
    """실시간 진행률 모니터링"""
    
    def __init__(self):
        self.metrics = {
            'generated': [],
            'passed': [],
            'quality_scores': [],
            'generation_speed': [],
            'chain_improvements': [],
            'timestamps': []
        }
        self.start_time = time.time()
        self.last_update = self.start_time
    
    def update(self, new_questions: List[Dict], total_generated: int, total_passed: int):
        """메트릭 업데이트"""
        current_time = time.time()
        
        # 생성 속도 계산 (분당)
        time_diff = (current_time - self.last_update) / 60
        if time_diff > 0:
            speed = len(new_questions) / time_diff
        else:
            speed = 0
        
        self.metrics['generated'].append(total_generated)
        self.metrics['passed'].append(total_passed)
        self.metrics['generation_speed'].append(speed)
        self.metrics['timestamps'].append(current_time - self.start_time)
        
        # 품질 점수 추가
        for q in new_questions:
            if 'metadata' in q and 'final_score' in q['metadata']:
                self.metrics['quality_scores'].append(q['metadata']['final_score'])
                
                # 체이닝 개선도
                if 'initial_score' in q['metadata']:
                    improvement = q['metadata']['final_score'] - q['metadata']['initial_score']
                    self.metrics['chain_improvements'].append(improvement)
        
        self.last_update = current_time
    
    def plot_dashboard(self):
        """대시보드 플롯"""
        clear_output(wait=True)
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('FSKU 데이터 생성 모니터링 대시보드', fontsize=16)
        
        # 1. 생성 진행률
        if self.metrics['generated']:
            axes[0, 0].plot(self.metrics['timestamps'], self.metrics['generated'], 'b-', linewidth=2)
            axes[0, 0].fill_between(self.metrics['timestamps'], 0, self.metrics['generated'], alpha=0.3)
            axes[0, 0].set_title('누적 생성 문제 수')
            axes[0, 0].set_xlabel('시간 (초)')
            axes[0, 0].set_ylabel('문제 수')
            axes[0, 0].grid(True, alpha=0.3)
        
        # 2. 통과율
        if self.metrics['generated'] and self.metrics['passed']:
            pass_rates = [p/g*100 if g > 0 else 0 for p, g in zip(self.metrics['passed'], self.metrics['generated'])]
            axes[0, 1].plot(self.metrics['timestamps'], pass_rates, 'g-', linewidth=2)
            axes[0, 1].axhline(y=config['min_quality_score'], color='r', linestyle='--', label=f"목표: {config['min_quality_score']}%")
            axes[0, 1].set_title('품질 통과율 (%)')
            axes[0, 1].set_xlabel('시간 (초)')
            axes[0, 1].set_ylabel('통과율 (%)')
            axes[0, 1].set_ylim(0, 100)
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
        
        # 3. 생성 속도
        if self.metrics['generation_speed']:
            axes[0, 2].plot(self.metrics['timestamps'], self.metrics['generation_speed'], 'r-', linewidth=2)
            axes[0, 2].set_title('생성 속도 (문제/분)')
            axes[0, 2].set_xlabel('시간 (초)')
            axes[0, 2].set_ylabel('속도')
            axes[0, 2].grid(True, alpha=0.3)
        
        # 4. 품질 점수 분포
        if self.metrics['quality_scores']:
            axes[1, 0].hist(self.metrics['quality_scores'], bins=20, alpha=0.7, color='purple')
            axes[1, 0].axvline(x=config['min_quality_score'], color='r', linestyle='--', label=f"최소: {config['min_quality_score']}")
            axes[1, 0].set_title('품질 점수 분포')
            axes[1, 0].set_xlabel('점수')
            axes[1, 0].set_ylabel('빈도')
            axes[1, 0].legend()
        
        # 5. 체이닝 개선도
        if self.metrics['chain_improvements']:
            axes[1, 1].hist(self.metrics['chain_improvements'], bins=20, alpha=0.7, color='orange')
            avg_improvement = np.mean(self.metrics['chain_improvements'])
            axes[1, 1].axvline(x=avg_improvement, color='g', linestyle='--', label=f"평균: {avg_improvement:.1f}")
            axes[1, 1].set_title('체이닝 품질 개선도')
            axes[1, 1].set_xlabel('개선 점수')
            axes[1, 1].set_ylabel('빈도')
            axes[1, 1].legend()
        
        # 6. 요약 통계
        axes[1, 2].axis('off')
        if self.metrics['generated']:
            total_time = (time.time() - self.start_time) / 60  # 분
            stats_text = f"""📊 실시간 통계
            
총 생성: {self.metrics['generated'][-1]:,}개
통과: {self.metrics['passed'][-1]:,}개
통과율: {self.metrics['passed'][-1]/self.metrics['generated'][-1]*100:.1f}%

경과 시간: {total_time:.1f}분
평균 속도: {self.metrics['generated'][-1]/total_time:.1f}개/분

평균 품질: {np.mean(self.metrics['quality_scores']):.1f}점
최고 품질: {max(self.metrics['quality_scores']):.1f}점
최저 품질: {min(self.metrics['quality_scores']):.1f}점
"""
            axes[1, 2].text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center')
        
        plt.tight_layout()
        plt.show()
        
        # 예상 완료 시간
        if self.metrics['generated'] and self.metrics['generated'][-1] > 0:
            current_rate = self.metrics['generated'][-1] / (time.time() - self.start_time)
            remaining = config['num_questions'] - self.metrics['generated'][-1]
            eta_seconds = remaining / current_rate if current_rate > 0 else 0
            eta_minutes = eta_seconds / 60
            
            print(f"\n⏱️ 예상 완료 시간: {eta_minutes:.1f}분 후")
            print(f"🎯 목표 달성률: {self.metrics['generated'][-1]/config['num_questions']*100:.1f}%")

## 7. 대규모 배치 생성 함수

In [ ]:
def generate_large_dataset(config: Dict[str, Any], retriever: DocumentRetriever) -> List[Dict]:
    """대규모 데이터셋 생성"""
    print("="*60)
    print("🚀 대규모 데이터 생성 시작")
    print(f"📋 설정:")
    print(f"  - 목표: {config['num_questions']:,}개")
    print(f"  - 모델: {config['model_name']}")
    print(f"  - 배치 크기: {config['batch_size']}")
    print(f"  - 체이닝: {config['use_chaining']}")
    print(f"  - 최소 품질: {config['min_quality_score']}점")
    print("="*60)
    
    # 생성기 초기화
    generator = ChainDataGenerator(config)
    monitor = ProgressMonitor()
    
    # 결과 저장
    all_questions = []
    all_metadata = []
    checkpoint_counter = 0
    
    # 청크 가져오기
    all_chunks = retriever.chunks
    print(f"\n📚 사용 가능한 청크: {len(all_chunks)}개")
    
    # 청크를 순환하며 사용
    chunk_index = 0
    total_generated = 0
    total_passed = 0
    
    try:
        with tqdm(total=config['num_questions'], desc="전체 진행률") as pbar:
            while len(all_questions) < config['num_questions']:
                # 배치 크기만큼 청크 선택
                batch_chunks = []
                for _ in range(config['batch_size']):
                    batch_chunks.append(all_chunks[chunk_index % len(all_chunks)])
                    chunk_index += 1
                
                # 배치 생성
                batch_start_time = time.time()
                batch_questions = generator.generate_questions_batch(
                    chunks=batch_chunks,
                    batch_size=config['batch_size']
                )
                batch_time = time.time() - batch_start_time
                
                # 품질 필터링
                passed_questions = []
                for q in batch_questions:
                    total_generated += 1
                    
                    # 품질 검사
                    if 'metadata' in q and q['metadata'].get('final_score', 0) >= config['min_quality_score']:
                        passed_questions.append(q)
                        total_passed += 1
                    elif 'metadata' not in q:  # 기존 방식 생성
                        # 간단한 품질 체크
                        if len(q.get('question', '')) > 20 and len(q.get('answer', '')) > 0:
                            passed_questions.append(q)
                            total_passed += 1
                
                # 결과 추가
                all_questions.extend(passed_questions)
                pbar.update(len(passed_questions))
                
                # 모니터링 업데이트
                monitor.update(passed_questions, total_generated, total_passed)
                
                # 대시보드 업데이트 (설정된 간격마다)
                if total_generated % config['update_interval'] == 0 and config['enable_monitoring']:
                    monitor.plot_dashboard()
                
                # 체크포인트 저장
                if len(all_questions) % config['checkpoint_interval'] == 0 and len(all_questions) > 0:
                    checkpoint_counter += 1
                    checkpoint_file = config['output_dir'] / f'checkpoint_{checkpoint_counter}_{len(all_questions)}.jsonl'
                    
                    with open(checkpoint_file, 'w', encoding='utf-8') as f:
                        for q in all_questions[-config['checkpoint_interval']:]:
                            f.write(json.dumps(q, ensure_ascii=False) + '\n')
                    
                    print(f"\n💾 체크포인트 저장: {checkpoint_file.name}")
                    print(f"   배치 처리 시간: {batch_time:.1f}초")
                    print(f"   배치 통과율: {len(passed_questions)/len(batch_questions)*100:.1f}%")
                
                # 목표 달성 시 종료
                if len(all_questions) >= config['num_questions']:
                    all_questions = all_questions[:config['num_questions']]
                    break
    
    except KeyboardInterrupt:
        print("\n⚠️ 사용자에 의해 중단됨")
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        # 최종 대시보드 표시
        if config['enable_monitoring']:
            monitor.plot_dashboard()
        
        # 메모리 정리
        generator.cleanup()
    
    print(f"\n✅ 생성 완료!")
    print(f"  - 총 시도: {total_generated:,}개")
    print(f"  - 통과: {len(all_questions):,}개")
    print(f"  - 통과율: {len(all_questions)/total_generated*100:.1f}%")
    
    return all_questions


def save_final_dataset(questions: List[Dict], config: Dict[str, Any]):
    """최종 데이터셋 저장"""
    print("\n📁 최종 데이터셋 저장 중...")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 전체 데이터 JSONL 저장
    full_file = config['output_dir'] / f'full_dataset_{timestamp}.jsonl'
    with open(full_file, 'w', encoding='utf-8') as f:
        for q in questions:
            f.write(json.dumps(q, ensure_ascii=False) + '\n')
    print(f"✅ 전체 데이터: {full_file.name}")
    
    # 2. 훈련/검증/테스트 분할
    from sklearn.model_selection import train_test_split
    
    # 80/10/10 분할
    train_val, test = train_test_split(questions, test_size=0.1, random_state=42)
    train, val = train_test_split(train_val, test_size=0.111, random_state=42)  # 0.111 ≈ 10/90
    
    # 각각 저장
    splits = {
        'train': train,
        'val': val,
        'test': test
    }
    
    for split_name, split_data in splits.items():
        split_file = config['output_dir'] / f'{split_name}_{timestamp}.jsonl'
        with open(split_file, 'w', encoding='utf-8') as f:
            for q in split_data:
                f.write(json.dumps(q, ensure_ascii=False) + '\n')
        print(f"✅ {split_name}: {split_file.name} ({len(split_data):,}개)")
    
    # 3. 요약 리포트 생성
    report = {
        'generation_info': {
            'timestamp': timestamp,
            'total_questions': len(questions),
            'config': config
        },
        'splits': {
            'train': len(train),
            'val': len(val),
            'test': len(test)
        },
        'sample_questions': questions[:5]  # 샘플 5개
    }
    
    report_file = config['output_dir'] / f'generation_report_{timestamp}.json'
    with open(report_file, 'w', encoding='utf-8') as f:
        json.dump(report, f, ensure_ascii=False, indent=2)
    print(f"✅ 리포트: {report_file.name}")
    
    # 4. 샘플 출력
    print("\n📝 생성된 문제 샘플:")
    for i, q in enumerate(questions[:3], 1):
        print(f"\n[{i}] ID: {q['id']}")
        print(f"질문: {q['question'][:100]}..." if len(q['question']) > 100 else f"질문: {q['question']}")
        print(f"답변: {q['answer'][:50]}..." if len(q['answer']) > 50 else f"답변: {q['answer']}")
        if 'metadata' in q:
            print(f"품질: {q['metadata'].get('final_score', 'N/A')}점")

## 8. 메인 실행

In [ ]:
# 메인 실행
if 'retriever' in locals():
    # 데이터 생성 실행
    questions = generate_large_dataset(config, retriever)
    
    # 결과 저장
    if questions:
        save_final_dataset(questions, config)
else:
    print("⚠️ RAG 시스템이 초기화되지 않았습니다.")
    print("Cell 4를 먼저 실행하여 RAG를 초기화하세요.")

## 9. 결과 분석 및 통계

In [ ]:
# 생성된 데이터 분석
def analyze_generated_data(output_dir: Path):
    """생성된 데이터 분석"""
    # 최신 파일 찾기
    jsonl_files = list(output_dir.glob('full_dataset_*.jsonl'))
    if not jsonl_files:
        print("⚠️ 생성된 데이터 파일이 없습니다.")
        return
    
    latest_file = max(jsonl_files, key=lambda f: f.stat().st_mtime)
    print(f"📊 분석 파일: {latest_file.name}")
    
    # 데이터 로드
    questions = []
    with open(latest_file, 'r', encoding='utf-8') as f:
        for line in f:
            questions.append(json.loads(line))
    
    print(f"\n📈 데이터 통계:")
    print(f"  - 총 문제 수: {len(questions):,}개")
    
    # 문제 길이 분석
    q_lengths = [len(q['question']) for q in questions]
    a_lengths = [len(q['answer']) for q in questions]
    
    print(f"\n📏 문제 길이:")
    print(f"  - 평균: {np.mean(q_lengths):.1f}자")
    print(f"  - 최소: {min(q_lengths)}자")
    print(f"  - 최대: {max(q_lengths)}자")
    
    print(f"\n📏 답변 길이:")
    print(f"  - 평균: {np.mean(a_lengths):.1f}자")
    print(f"  - 최소: {min(a_lengths)}자")
    print(f"  - 최대: {max(a_lengths)}자")
    
    # 메타데이터 분석
    has_metadata = sum(1 for q in questions if 'metadata' in q)
    print(f"\n🏷️ 메타데이터:")
    print(f"  - 메타데이터 포함: {has_metadata}개 ({has_metadata/len(questions)*100:.1f}%)")
    
    if has_metadata > 0:
        scores = [q['metadata'].get('final_score', 0) for q in questions if 'metadata' in q and 'final_score' in q['metadata']]
        if scores:
            print(f"\n🎯 품질 점수:")
            print(f"  - 평균: {np.mean(scores):.1f}점")
            print(f"  - 표준편차: {np.std(scores):.1f}")
            print(f"  - 최소: {min(scores):.1f}점")
            print(f"  - 최대: {max(scores):.1f}점")
    
    # 시각화
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 문제 길이 분포
    axes[0].hist(q_lengths, bins=30, alpha=0.7, color='blue')
    axes[0].set_title('문제 길이 분포')
    axes[0].set_xlabel('글자 수')
    axes[0].set_ylabel('빈도')
    
    # 답변 길이 분포
    axes[1].hist(a_lengths, bins=30, alpha=0.7, color='green')
    axes[1].set_title('답변 길이 분포')
    axes[1].set_xlabel('글자 수')
    axes[1].set_ylabel('빈도')
    
    # 품질 점수 분포
    if scores:
        axes[2].hist(scores, bins=20, alpha=0.7, color='red')
        axes[2].set_title('품질 점수 분포')
        axes[2].set_xlabel('점수')
        axes[2].set_ylabel('빈도')
    else:
        axes[2].text(0.5, 0.5, '품질 점수 없음', ha='center', va='center')
        axes[2].set_xlim(0, 1)
        axes[2].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()

# 분석 실행
analyze_generated_data(config['output_dir'])

## 10. 추가 유틸리티

In [ ]:
# 데이터 병합 유틸리티
def merge_checkpoints(output_dir: Path) -> str:
    """체크포인트 파일들을 병합"""
    checkpoint_files = sorted(output_dir.glob('checkpoint_*.jsonl'))
    
    if not checkpoint_files:
        print("체크포인트 파일이 없습니다.")
        return None
    
    print(f"\n🔀 {len(checkpoint_files)}개 체크포인트 병합 중...")
    
    all_questions = []
    seen_ids = set()
    
    for file in checkpoint_files:
        with open(file, 'r', encoding='utf-8') as f:
            for line in f:
                q = json.loads(line)
                if q['id'] not in seen_ids:
                    all_questions.append(q)
                    seen_ids.add(q['id'])
    
    # 병합 파일 저장
    merged_file = output_dir / f'merged_{datetime.now().strftime("%Y%m%d_%H%M%S")}.jsonl'
    with open(merged_file, 'w', encoding='utf-8') as f:
        for q in all_questions:
            f.write(json.dumps(q, ensure_ascii=False) + '\n')
    
    print(f"✅ 병합 완료: {merged_file.name}")
    print(f"   총 {len(all_questions):,}개 문제 (중복 제거됨)")
    
    return str(merged_file)


# 품질 필터링 유틸리티
def filter_by_quality(input_file: Path, min_score: float = 85.0) -> str:
    """품질 점수로 필터링"""
    questions = []
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            q = json.loads(line)
            if 'metadata' in q and q['metadata'].get('final_score', 0) >= min_score:
                questions.append(q)
    
    # 필터링된 파일 저장
    filtered_file = input_file.parent / f'filtered_{min_score}_{input_file.name}'
    with open(filtered_file, 'w', encoding='utf-8') as f:
        for q in questions:
            f.write(json.dumps(q, ensure_ascii=False) + '\n')
    
    print(f"✅ 필터링 완료: {filtered_file.name}")
    print(f"   {len(questions):,}개 문제 (최소 {min_score}점)")
    
    return str(filtered_file)


# HuggingFace 형식 변환
def convert_to_hf_format(input_file: Path) -> pd.DataFrame:
    """HuggingFace datasets 형식으로 변환"""
    data = []
    
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            q = json.loads(line)
            data.append({
                'instruction': q['question'],
                'output': q['answer'],
                'id': q['id']
            })
    
    df = pd.DataFrame(data)
    
    # Parquet 형식으로 저장 (HF 권장)
    parquet_file = input_file.parent / f'{input_file.stem}.parquet'
    df.to_parquet(parquet_file, index=False)
    
    print(f"✅ HF 형식 변환 완료: {parquet_file.name}")
    print(f"   Shape: {df.shape}")
    
    return df


# 유틸리티 메뉴
print("\n🛠️ 추가 유틸리티:")
print("1. merge_checkpoints(config['output_dir']) - 체크포인트 병합")
print("2. filter_by_quality(file_path, min_score) - 품질 필터링")
print("3. convert_to_hf_format(file_path) - HuggingFace 형식 변환")